In [11]:
import os
from openai import OpenAI
client = OpenAI()

client.api_key = os.getenv("OPENAI_API_KEY")

In [12]:
assistant = client.beta.assistants.create(
    name="국어 선생님",
    instructions="당신은 친절한 국어선생님입니다. 사용자가 질문하는 국어문제에 대답하기 위해 retrieval을 실행하세요.",
    tools=[{"type": "retrieval"}],
    model="gpt-3.5-turbo"
)
assistant

Assistant(id='asst_BYVPaGEUCC4MpQAVKUlaEb38', created_at=1710317255, description=None, file_ids=[], instructions='당신은 친절한 국어선생님입니다. 사용자가 질문하는 국어문제에 대답하기 위해 retrieval을 실행하세요.', metadata={}, model='gpt-3.5-turbo', name='국어 선생님', object='assistant', tools=[ToolRetrieval(type='retrieval')])

In [13]:
file = client.files.create(
  file=open("소나기.pdf", "rb"),
  purpose='assistants'
)

In [14]:
thread = client.beta.threads.create()
thread

Thread(id='thread_KxIiDdDol0EnbjVSRYGbuJAj', created_at=1710317263, metadata={}, object='thread')

In [15]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="소년과 소녀는 어디에서 처음 만났나?",
    file_ids=[file.id]
)

In [16]:
thread_messages = client.beta.threads.messages.list(thread.id)
print(thread_messages.data)

[ThreadMessage(id='msg_tXZpQBWbXwdnbBeOSL0ug9Kn', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='소년과 소녀는 어디에서 처음 만났나?'), type='text')], created_at=1710317265, file_ids=['file-e513Hjiq3vKxUUbaNRFei9NS'], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_KxIiDdDol0EnbjVSRYGbuJAj')]


In [17]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="주어진 질문에 대해 답을 줘. 답을 모르겠으면 모르겠다고 답해야 하고 답을 지어내면 안돼."
)

In [18]:
import time

while run.status != "completed"  :
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )
    time.sleep(0.1)

In [19]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

In [20]:
thread_messages = client.beta.threads.messages.list(thread.id)
print(thread_messages.data)

[ThreadMessage(id='msg_Y92vwhEM8tbjuUnIlmX5BEKe', assistant_id='asst_BYVPaGEUCC4MpQAVKUlaEb38', content=[MessageContentText(text=Text(annotations=[TextAnnotationFileCitation(end_index=122, file_citation=TextAnnotationFileCitationFileCitation(file_id='file-e513Hjiq3vKxUUbaNRFei9NS', quote=''), start_index=112, text='【6†source】', type='file_citation')], value='소년과 소녀는 이야기 속에서 처음으로 개울가에서 만났습니다. 소년은 개울가에서 소녀를 보고 곧 윤 초시네의 증손녀(曾孫女)딸이라는 걸 알아차렸고, 소녀는 개울가에서 손을 잠그고 물장난을 하고 있었습니다【6†source】.'), type='text')], created_at=1710317274, file_ids=[], metadata={}, object='thread.message', role='assistant', run_id='run_biUq5BmhtQe8QXNNKvC4DZGp', thread_id='thread_KxIiDdDol0EnbjVSRYGbuJAj'), ThreadMessage(id='msg_vtW9oo0otf1mDaK24V0ECJSR', assistant_id='asst_BYVPaGEUCC4MpQAVKUlaEb38', content=[MessageContentText(text=Text(annotations=[], value='제가 업로드된 파일을 확인할게요. 잠시 기다려 주세요.'), type='text')], created_at=1710317272, file_ids=[], metadata={}, object='thread.message', role='assistant', run_id='run_biUq5BmhtQe

In [22]:
print(thread_messages.data[0].role)
print(thread_messages.data[0].content[0].text.value)

assistant
소년과 소녀는 이야기 속에서 처음으로 개울가에서 만났습니다. 소년은 개울가에서 소녀를 보고 곧 윤 초시네의 증손녀(曾孫女)딸이라는 걸 알아차렸고, 소녀는 개울가에서 손을 잠그고 물장난을 하고 있었습니다【6†source】.
